In [0]:
# Interaction_5_6(Solutions to Exercise 5.6)
# Author: Matthew Dixon
# Version: 1.0 (08.09.2019)
# License: MIT
# Email: matthew.dixon@iit.edu
# Notes: tested on Mac OS X with Python 3.6
# Citation: Please cite the following reference if this notebook is used for research purposes:
# Dixon M.F. I. Halperin and P. Bilokon, Machine Learning in Finance: From Theory to Practice, Springer Graduate textbook Series, 2020. 
# Solution courtesy of Shengnan Li

In [2]:
import random
import pandas as pd
import numpy as np
import seaborn as sns
import math
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.regularizers import l1,l2
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_val_score, train_test_split
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, mean_absolute_error
import statsmodels.api as sm
import statsmodels.stats.diagnostic as tds
from statsmodels.api import add_constant
from scipy import stats


Using TensorFlow backend.


# Simple Data Generation Process (DGP)

$Y=X_1+X_2 + X_1X_2+\epsilon, ~X_1, X_2 \sim N(0,1,), \epsilon \sim N(0,\sigma_n^2)$

In [0]:
M = 5000
np.random.seed(7)
X = np.zeros(shape=(M,2))
sigma_n = 0.01
X[:int(M/2),0]= np.random.randn(int(M/2))
X[:int(M/2),1]= np.random.randn(int(M/2))

X[int(M/2):,0]= -X[:int(M/2),0]
X[int(M/2):,1]= -X[:int(M/2),1]

eps= np.random.randn(M)
Y= 1.0*X[:,0] + 1.0*X[:,1] + 1.0*X[:,0]*X[:,1] + sigma_n*eps

In [0]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)

# Compare with a FFW Neural Network with one hidden layer (tanh activated)

From running the code, it is evident that increasing the number of hidden units improves the interpretability of the interaction term.  Put another way, as the number of hidden units increases, the mean of the true interaction term converges to 1, and the standard deviation of the interaction term decreases. Also, the mean of the false interaction terms converge to zero with increasing confidence. 

For reference, below is a table that shows the mean and standard deviation of the interaction term as the number of hidden units is increased.  This table only includes one hidden layer with tanh activation.

## Compute the sensitivities

In [0]:
# Assume that the activation function is tanh
def sensitivities(lm, X):
    
    W1=lm.model.get_weights()[0]
    b1=lm.model.get_weights()[1]
    W2=lm.model.get_weights()[2]
    b2=lm.model.get_weights()[3]
    
    
    M = np.shape(X)[0]
    p = np.shape(X)[1]

    beta=np.array([0]*M*(p+1), dtype='float32').reshape(M,p+1)
    beta_interact=np.array([0]*M*p*p, dtype='float32').reshape(M,p,p)
    
    beta[:,0]= (np.dot(np.transpose(W2),np.tanh(b1)) + b2)[0] # intercept \beta_0= F_{W,b}(0)
    for i in range(M):
 
      Z1 = np.tanh(np.dot(np.transpose(W1),np.transpose(X[i,])) + b1)
      
      D = np.diag(1-Z1**2) 
      D_prime =np.diag(-2*Z1*(1-Z1**2))   # needed for interaction term     
      #D = np.diag(np.sign(Z1))  
        
      for j in range(p):  
          beta[i,j+1]=np.dot(np.transpose(W2),np.dot(D,W1[j]))
          #interaction term 
          for k in range(p):
            beta_interact[i,j,k]=np.dot(np.transpose(W2),np.dot(np.diag(W1[j]), np.dot(D_prime,W1[k]))) # Hessian matrix  
    
            
    return(beta, beta_interact)

## Increasing the number of hidden units

In [0]:
# number of hidden neurons
N= [10,20,50,100,200]

In [56]:
array_mean_inter = []
array_std_inter = []
for k in range(len(N)):
  n = N[k]
  # with non-linear activation
  def linear_NN1_model_act(l1_reg=0.0):    
      model = Sequential()
      model.add(Dense(n, input_dim=2, kernel_initializer='normal', activation='tanh'))
      model.add(Dense(1, kernel_initializer='normal')) 
      model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
      return model
    
  lm = KerasRegressor(build_fn=linear_NN1_model_act, epochs=100, batch_size=10, verbose=0, callbacks=[es])
  lm.fit(X,Y)
  beta, beta_inter=sensitivities(lm, X)
  array_mean_inter.append(int(n))
  array_mean_inter.extend(np.concatenate(np.mean(beta_inter, axis=0).tolist()))
  array_std_inter.append(int(n))
  array_std_inter.extend(np.concatenate(np.std(beta_inter, axis=0).tolist()))
  
pd_mean_inter = pd.DataFrame(np.reshape(array_mean_inter,(len(N),5)),columns=['number_of_neurons','H11','H12','H21','H22'])
pd_std_inter = pd.DataFrame(np.reshape(array_std_inter,(len(N),5)),columns=['number_of_neurons','H11','H12','H21','H22'])
print(pd_mean_inter)
print(pd_std_inter)

   number_of_neurons       H11       H12       H21       H22
0               10.0 -0.026601  0.970886  0.970886 -0.018296
1               20.0  0.010413  1.029390  1.029390  0.016088
2               50.0  0.003500  1.005061  1.005061 -0.000847
3              100.0 -0.002594  0.994979  0.994979 -0.001575
4              200.0  0.001534  0.998311  0.998311 -0.002615
   number_of_neurons       H11       H12       H21       H22
0               10.0  0.277478  0.266668  0.266668  0.254695
1               20.0  0.094886  0.090346  0.090346  0.090399
2               50.0  0.079534  0.081473  0.081473  0.085492
3              100.0  0.053773  0.055478  0.055478  0.059201
4              200.0  0.057174  0.059285  0.059285  0.063547
